In [1]:
#%%script echo 'ignore cell'
%pip install ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive, files

import pandas as pd
import numpy  as np
import warnings

drive.mount('/content/drive', force_remount=True)

warnings.filterwarnings('ignore')
%load_ext autotime

Mounted at /content/drive
time: 630 µs (started: 2024-04-03 23:30:42 +00:00)


In [4]:
train = pd.read_parquet('drive/MyDrive/Colab Notebooks/datasets/enefit/custom/train_featured_full.parquet')
test  = pd.read_parquet('drive/MyDrive/Colab Notebooks/datasets/enefit/custom/test_featured_full.parquet')

time: 14.3 s (started: 2024-04-03 23:30:44 +00:00)


In [6]:
reference = train[['puid', 'cons', 'prod', 'dt']]
train.drop(['puid', 'cons', 'prod', 'dt'], axis=1, inplace=True)
test.drop(['puid', 'cons', 'prod', 'dt'], axis=1, inplace=True)

time: 1.24 s (started: 2024-04-03 23:31:50 +00:00)


In [7]:
label = ['consumption', 'production']
y_train = train[label]
X_train = train.drop(label, axis=1)

y_test = test[label]
X_test = test.drop(label, axis=1)

time: 1.09 s (started: 2024-04-03 23:32:06 +00:00)


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

ss = StandardScaler()
y_train = ss.fit_transform(y_train)

time: 807 ms (started: 2024-04-03 18:01:31 +00:00)


# Model Training

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
colors = sns.color_palette()
colors

[(0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (1.0, 0.4980392156862745, 0.054901960784313725),
 (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
 (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
 (0.5803921568627451, 0.403921568627451, 0.7411764705882353),
 (0.5490196078431373, 0.33725490196078434, 0.29411764705882354),
 (0.8901960784313725, 0.4666666666666667, 0.7607843137254902),
 (0.4980392156862745, 0.4980392156862745, 0.4980392156862745),
 (0.7372549019607844, 0.7411764705882353, 0.13333333333333333),
 (0.09019607843137255, 0.7450980392156863, 0.8117647058823529)]

time: 975 ms (started: 2024-04-03 23:32:51 +00:00)


In [12]:
from sklearn.ensemble     import ExtraTreesRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics      import mean_absolute_error
from sklearn.multioutput  import MultiOutputRegressor
from sklearn.tree         import DecisionTreeRegressor

import lightgbm as lgb

time: 2 s (started: 2024-04-03 23:32:58 +00:00)


In [18]:
def mae(model, X_train, y_train, X_test, y_test):
  y_pred = model.predict(X_train)
  print(f'Train MAE: {mean_absolute_error(y_train, y_pred)}')

  y_pred = model.predict(X_test)
  print(f'Test  MAE: {mean_absolute_error(y_test, y_pred)}')

time: 1.07 ms (started: 2024-04-03 23:37:49 +00:00)


## ElasticNet

In [17]:
#%%script echo 'ignore cell'
en = ElasticNet()
en.fit(X_train, y_train)
mae(en, X_train, y_train, X_test, y_test)

Train MAE: 0.30158487505137743
Test MAE:  0.6243679245253075
time: 13.7 s (started: 2024-04-03 23:37:09 +00:00)


## DecisionTreeRegressor

In [30]:
#%%script echo 'ignore cell'
dr = DecisionTreeRegressor(max_depth=5, max_leaf_nodes=20)
dr.fit(X_train, y_train)
mae(dr, X_train, y_train, X_test, y_test)

Train MAE: 0.10611542889729038
Test  MAE: 0.27583246148235285
time: 2min 25s (started: 2024-04-04 00:37:46 +00:00)


## ExtraTreesRegressor

In [23]:
#%%script echo 'ignore cell'
et = ExtraTreesRegressor(n_estimators=5)
et.fit(X_train, y_train)
mae(et, X_train, y_train, X_test, y_test)

Train MAE: 1.0158540934604254e-17
Test  MAE: 0.1959169580553752
time: 11min 55s (started: 2024-04-04 00:08:25 +00:00)


# LGBMRegressor

In [19]:
#%%script echo 'ignore cell'
lgbm = MultiOutputRegressor(lgb.LGBMRegressor(n_estimators=100))
lgbm.fit(X_train.to_numpy(), y_train)
mae(lgbm, X_train, y_train, X_test, y_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.981196 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46336
[LightGBM] [Info] Number of data points in the train set: 987460, number of used features: 198
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.920202 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46336
[LightGBM] [Info] Number of data points in the train set: 987460, number of used features: 198
[LightGBM] [Info] Start training from score 0.000000
Train MAE: 0.02927075862929076
Test  MAE: 0.1650968402620491
time: 3min 40s (started: 2024-04-03 23:38:26 +00:00)
